In [21]:
#Importar librerías necesarias:

#Librería para manipulación y análisis de datos.
import pandas as pd

#Selenium para automatizar la interacción con navegadores web.
from selenium import webdriver

#Maneja la instalación automática del ChromeDriver.
from webdriver_manager.chrome import ChromeDriverManager

#Permite localizar elementos HTML usando diferentes métodos.
from selenium.webdriver.common.by import By

#Espera explícita para asegurar que los elementos se carguen.
from selenium.webdriver.support.ui import WebDriverWait 

#Condiciones de espera explícitas.
from selenium.webdriver.support import expected_conditions as EC

#Pausa la ejecución del código por un número de segundos.
from time import sleep

#Configuraciones iniciales para pandas:

#Configuración para mostrar todas las columnas del DataFrame.
pd.set_option('display.max_columns', None)

In [22]:
#Función para generar el driver de Selenium:

def generar_driver():
    #Configuración del navegador:
    driver = webdriver.Chrome() #Inicializa ChromeDriver.

    sleep(2)

    driver.maximize_window() #Maximiza la ventana del navegador.
    print("Maximizando la web.")

    sleep(2)

    driver.get("https://www.imdb.com/") #Navega a IMDb.
    print("Accediendo a la web.")

    sleep(2)
    
    #Aceptar cookies si aparece el botón.
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    print("Cookies aceptadas.")
    
    return driver

In [23]:
#Función para extraer información utilizando una lista de selectores CSS:

def extraer_con_selectores(driver, selectores):
    #Itera sobre los selectores proporcionados hasta encontrar uno que funcione.

    for selector in selectores:
        try:
            elemento = driver.find_element(By.CSS_SELECTOR, selector) #Busca el elemento usando CSS selector.
            if elemento: #Si el elemento se encuentra, devuelve su texto.
                return elemento.text
        except:
            continue #Si falla, pasa al siguiente selector.
    return "No disponible" #Devuelve "No disponible" si ninguno de los selectores funciona.

En la siguiente función usamos XPath.
 XPath es un lenguaje para seleccionar nodos en un documento XML/HTML. 
 Es especialmente útil para encontrar elementos en estructuras complejas de HTML que no pueden ser fácilmente seleccionadas con CSS selectors.

In [24]:
# Función para extraer los nombres de directores y guionistas filtrando correctamente

def extraer_nombres(driver, xpath_selector):
    try:
        #Encuentra los elementos usando el XPath especificado.
        elementos = driver.find_elements(By.XPATH, xpath_selector)
        #Limpia los textos, eliminando espacios adicionales y evitando elementos vacíos.
        nombres = [elemento.text.strip() for elemento in elementos if elemento.text.strip()]
        #Une los nombres en una cadena, separados por comas.
        return ", ".join(nombres).rstrip(",") if nombres else "No disponible"
    except:
        return "No disponible"

In [25]:
#Función principal para extraer los datos de una película dada su ID de IMDb.

def extraer_datos_pelicula(driver, id_pelicula):
#Construye la URL de la película usando su ID.
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}"
    driver.get(url_pelicula) #Navega a la página de la película.
    print(f"Accediendo a la película con ID {id_pelicula}")
    sleep(2)

    #Extrae nombre de la película.
    try:
        titulo_pelicula = driver.find_element(By.CSS_SELECTOR, 'h1').text #Busca el título usando un selector CSS simple.
        print("Nombre de la película:", titulo_pelicula)
    except:
        titulo_pelicula = "No disponible"
        print("Nombre de la película:", titulo_pelicula)

    #Lista de posibles selectores para la puntuación de IMDb.
    selectores_puntuacion = [
        'span[class*="AggregateRatingButton__RatingScore"]',
        'span[data-testid="hero-rating-bar__aggregate-rating__score"]',
        'div[data-testid="hero-rating-bar__aggregate-rating__score"] span'
    ]

    #Intenta extraer la puntuación utilizando varios selectores.
    puntuacion_pelicula = extraer_con_selectores(driver, selectores_puntuacion)
    print("Puntuación de IMDb:", puntuacion_pelicula)

    #Extrae los directores utilizando un selector XPath específico.
    directores_texto = extraer_nombres(driver, '//a[contains(@href, "/name/") and contains(@href, "/?ref_=tt_ov_dr")]')
    print("Directores:", directores_texto)

    #Extrae los guionistas utilizando un selector XPath específico.
    guionistas_texto = extraer_nombres(driver, '//a[contains(@href, "/name/") and contains(@href, "/?ref_=tt_ov_wr")]')
    print("Guionistas:", guionistas_texto)

    #Extrae el ID de la película desde la URL actual.
    try:
        argumento_pelicula = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="plot-xl"]').text
        print("Argumento:", argumento_pelicula)
    except:
        argumento_pelicula = "No disponible"
        print("Argumento:", argumento_pelicula)

    #Extrae el ID de la película desde la URL actual.
    id_pelicula = driver.current_url.split('/')[4]
    print("ID de la película:", id_pelicula)

    sleep(2)

    #Crea una tupla con todos los datos extraídos para almacenarlos posteriormente.
    tupla_datos_pelicula = (titulo_pelicula, puntuacion_pelicula, directores_texto, guionistas_texto, argumento_pelicula, id_pelicula)

    #Imprime la tupla de resultados.
    print("Datos extraídos:", tupla_datos_pelicula)

    return tupla_datos_pelicula


In [26]:
#Función para extraer los IDs de las películas desde un archivo CSV:

def extractor_idowners(archivo_csv):
    df_pelis_API = pd.read_csv(archivo_csv) #Lee el archivo CSV con los IDs de las películas.
    lista_idowners = df_pelis_API['IdOwner'].tolist() #Convierte la columna de IDs a una lista.
    return lista_idowners

In [27]:
#Lista de IDs de las películas a extraer.
ids_peliculas = extractor_idowners('peliculas_api_desde_1995_a_2009.csv')

In [28]:
#Crear el driver de Selenium.
mi_driver = generar_driver()

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.


In [29]:
#Lista para almacenar los datos extraídos de todas las películas.
datos_peliculas = []

In [30]:
#Itera sobre cada ID de película y extrae los datos correspondientes.
for id_owner in ids_peliculas:
    mis_datos = extraer_datos_pelicula(mi_driver, id_owner)
    datos_peliculas.append(mis_datos)

Accediendo a la película con ID tt0100267
Nombre de la película: Nobat e Asheghi
Puntuación de IMDb: 6,4
Directores: Mohsen Makhmalbaf
Guionistas: Mohsen Makhmalbaf
Argumento: Part one: A brown-haired man finds out that his wife cheats on him with a blonde-haired man. He kills his rival and is condemned to the capital punishment. Gazale takes her own life. Part two: Same story. Same denouement, the only variant being that it is the blond-haired man who is the husband and the brown-haired one who is the lover. Part three: The story starts ov... Leer todo
ID de la película: tt0100267
Datos extraídos: ('Nobat e Asheghi', '6,4', 'Mohsen Makhmalbaf', 'Mohsen Makhmalbaf', 'Part one: A brown-haired man finds out that his wife cheats on him with a blonde-haired man. He kills his rival and is condemned to the capital punishment. Gazale takes her own life. Part two: Same story. Same denouement, the only variant being that it is the blond-haired man who is the husband and the brown-haired one who

In [31]:
#Cerrar el driver después de la extracción.
mi_driver.quit()

In [32]:
#Convierte la lista de datos extraídos a un DataFrame de pandas.
df_peliculas = pd.DataFrame(datos_peliculas, columns=["Título", "Puntuación IMDb", "Directores", "Guionistas", "Argumento", "ID"])

In [33]:
#Guarda el DataFrame en un archivo CSV.
df_peliculas.to_csv('data_peliculas_1995_2009.csv', index=False)
print("Archivo 'peliculas_ext_sel_1984.csv' creado exitosamente.")

Archivo 'peliculas_ext_sel_1984.csv' creado exitosamente.
